# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
import pickle
import json
import pandas as pd
import os
import numpy as np
with open('contributors_sample.json', 'r', encoding='UTF-8') as f:
    file = json.load(f)
print(file[0]['username'])
print(file[:5])

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
mail = list()
dom = list()
for k in range(0, len(file)):
    if not (file[k]['mail'] in mail):
        mail.append(file[k]['mail'])
for k in mail:
    s = k.split('@')
    s = '@' + s[1]
    if not (s in dom):
        dom.append(s)
print(dom)

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def search_by_username(username):
    try:
        for kl in file:
            for k, l in kl.items():
                if k == 'username' and l == username:
                    print(kl)
    except ValueError:
        print('Username отсутствует')
print(search_by_username('uhebert'))

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
count_f = 0
count_m = 0
for k in file:
    for j, q in k.items():
        if j == 'sex' and q == 'F':
            count_f += 1
        elif j == 'sex' and q == 'M':
            count_m += 1
print(f'Женщин {count_f}, мужчин {count_m}')

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
file1 = pd.DataFrame(file)
file1 = file1[['id', 'username', 'sex']]
print(file1)

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
file3 = pd.read_csv('recipes_sample.csv')
file4 = pd.read_json('contributors_sample.json')
count = 0
for k in range(0, len(file4)):
    if file3['id'][k] == file4['id'][k]:
        count += 1
    else:
        file4.loc[len(file4.index)] = file3.iloc[k]
print(len(file3) - count, file4)

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
jobs_s = []
lst = []
with open('contributors_sample.json') as f:
    contributors_s = json.load(f)
contributors_cpy = contributors_s.copy()
jobs = np.array(contributors_s['jobs'])
name = np.array(contributors_s['username'])
for i in jobs:
    jobs_s+=i
jobs_u = np.unique(jobs_s)
for i in jobs_u:
    for j in range(contributors_s.shape[0]):
        if i in contributors_s['jobs'][j]:
            lst.append([i,contributors_s['username'][j]])
df_j = pd.DataFrame(lst, columns=['jobs','name'])
dct = {i :df_j.loc[df_j['jobs']==i,'name'].tolist()for i in jobs_u}
dct

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open('job_people.json', 'w') as f:
    json.dump(job_people, f, sort_keys=False, indent=2)
with open('job_people.txt', 'wb') as f:
    pickle.dump(job_people, f)

if os.path.getsize('job_people.txt') > os.path.getsize('job_people.json'):
    print('Json больше')
else:
    print('Pickle больше')

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.txt', 'rb') as f:
    data = pickle.load(f)
    print(data)

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
import requests
from bs4 import BeautifulSoup
with open('steps_sample.xml') as f:
    ab_1 = BeautifulSoup(f, 'xml')
ids = ab_1.find_all('id')
steps = ab_1.find_all('steps')
dct_res = {}
for i in range(len(ids)):
    dct_res[ids[i].getText(strip=True)]=(steps[i].getText(',',strip=True)).split(',')
with open("steps_sample.json", "w") as f_1:
    json.dump(dct_res, f_1)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
dct_num = {}
for j in range(len(steps)):
    if (len(steps[j])) not in dct_num:
        dct_num[len(steps[j])]=[ids[j].getText(strip=True)]
    else:
         dct_num[len(steps[j])].append(ids[j].getText(strip=True))
dct_num

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
recipes = ab_1.find_all('recipe')
lst_1 = []
for i in recipes:
    try:
        if i.step["has_minutes"] or i.step["has_hours"]:
            lst_1.append(i.id.text)
    except:
        pass
lst_1[:100]

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
d={'id':[],'n_steps':[]}
for row in ab_1.find_all('recipe'):
    d['id'].append(int(row.id.text))
    d['n_steps'].append(len(row.find_all('step')))
df=pd.DataFrame(d)
df.n_steps=df.n_steps.astype('float64')
df

recipes=pd.read_csv('recipes_sample.csv')

df3_4=pd.merge(recipes,df,on='id')#в столбце n_steps_x указаны данные из таблицы recipes; в n_steps_y кол-во 
                              #шагов из файла файла steps_sample.xml 
df3_4['n_steps']=df3_4['n_steps_y']
df3_4 = df3_4.drop('n_steps_x',axis=1)
df3_4 = df3_4.drop('n_steps_y',axis=1)
df3_4

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
print((df3_4[df3_4['n_steps'].isna()==True]).size)
df3_4.n_steps = df3_4.n_steps.astype(int)

df3_4.to_csv('recipes_sample_with_filled_nsteps.csv', encoding='utf-8')
df3_4